# Step 1 - Setting up data environment 
------
The author attribute information ("nodes.json") was importated and loaded into a pandas Dataframe.

Networkx graph library was used to store the training data, which was parsed from a txt file


In [ ]:
import json
import pandas as pd
import networkx as nx


#Author attribitutes loaded

nodefile = open("nodes.json","r")
nodes = json.load(nodefile)

Attribute_dataframe = pd.DataFrame()
for i in nodes:
    Attribute_dataframe = Attribute_dataframe.append(i,ignore_index=True)
    

#fill NaNs in attributes database
Attribute_dataframe = Attribute_dataframe.fillna(0)
Attribute_dataframe= Attribute_dataframe.set_index("id")

    
    
G = nx.Graph()
for i in Attribute_dataframe.index:
    node = i
    G.add_node(int(node))

network_graph = open("train.txt","r")
network_graph = network_graph.readlines()

for i in network_graph:
    line = i.split(" ")
    node = line[0]
    edges = line[1]
    edges = edges.rstrip("\n")
    edges = edges.split("\t")
    for i in edges:
        G.add_edge(int(node),int(i))

#Step 2 - Creating negative edges set
----
Edges that are not present in the training graph are determined. The edges which have maximum hop length of 2 are removed from the edges set

In [ ]:
from itertools import combinations

comb = combinations(G.nodes,2)

missing_edges = []

for i in comb:
    if i not in G.edges:
        missing_edges.append(i)


missing_test_data = []
for i in missing_edges:
    try:
        if nx.shortest_path_length(G,source=i[0],target=i[1]) > 2:
            missing_test_data.append(i)
    except:
        print(str(i) + "has no path")

#Step 3 - Feature data development of the training set
----
23000 edges are randomly selected from the missing edge set. Network features are computed for each of the missing edges and the training data set is placed into a pandas DataFrame. 

Network features of the edges in the training set (positives) are calculated and the training data set is placed into a pandas DataFrame

In [ ]:
import random
from sklearn.metrics.pairwise import cosine_similarity

test_sample_size = 23000
missing_test_data_selected = random.sample(missing_test_data,test_sample_size)


network_data_table = pd.DataFrame()

count = 0

for i in missing_test_data_selected:
    print(count)
    source = i[0]
    sink = i[1]
    network_data_table = network_data_table.set_value(count,"Source",i[0])
    network_data_table = network_data_table.set_value(count,"Sink",i[1])
    array = [(source,sink)]
    preds = nx.resource_allocation_index(G,array)
    for u,v,p in preds:
        ra_score = p
    network_data_table = network_data_table.set_value(count,"ra_score",ra_score)
    preds = nx.jaccard_coefficient(G,array)
    for u,v,p in preds:
        jc_score = p
    network_data_table = network_data_table.set_value(count,"jc_score",jc_score)
    preds = nx.preferential_attachment(G,array)
    for u,v,p in preds:
        pa_score = p
    network_data_table = network_data_table.set_value(count,"pa_score",pa_score)
    preds = nx.adamic_adar_index(G,array)
    for u,v,p in preds:
        pa_score = p
    network_data_table = network_data_table.set_value(count,"aa_score",pa_score)
    cn_num = len(sorted(nx.common_neighbors(G, source, sink)))
    network_data_table = network_data_table.set_value(count,"cn_num",cn_num)
    network_data_table = network_data_table.set_value(count,"Class",0)
    count += 1

for i in G.edges():
    if i in G.edges():
        print(count)
        network_data_table = network_data_table.set_value(count,"Source",i[0])
        network_data_table = network_data_table.set_value(count,"Sink",i[1])
        source = i[0]
        sink = i[1]
        array = [(source,sink)]
        preds = nx.resource_allocation_index(G,array)
        for u,v,p in preds:
            ra_score = p
        network_data_table = network_data_table.set_value(count,"ra_score",ra_score)
        preds = nx.jaccard_coefficient(G,array)
        for u,v,p in preds:
            jc_score = p
        network_data_table = network_data_table.set_value(count,"jc_score",jc_score)
        preds = nx.preferential_attachment(G,array)
        for u,v,p in preds:
            pa_score = p
        network_data_table = network_data_table.set_value(count,"pa_score",pa_score)
        preds = nx.adamic_adar_index(G,array)
        for u,v,p in preds:
            pa_score = p
        network_data_table = network_data_table.set_value(count,"aa_score",pa_score)
        cn_num = len(sorted(nx.common_neighbors(G, source, sink)))
        network_data_table = network_data_table.set_value(count,"cn_num",cn_num)
        network_data_table = network_data_table.set_value(count,"AClass",1)
        count += 1
        

#Step 4 - Feature data development of the training set
----
The data is transformed transformed from a pandas dataframe to a numpy array, shuffled and then in to seperate arrays for x and y values.

In [ ]:
network_values_table = network_values_table.drop(["Source","Sink"],axis=1)
network_values_table = network_values_table[sorted(network_values_table.columns)]
values = network_values_table.values
np.random.shuffle(values)

y = np.empty([training_sample_size+test_sample_size,1])
count = 0
x_value = np.empty([training_sample_size+test_sample_size,5])
for i in values:
    y[count] = i[0]
    x_value[count] = i[1:0]
    count += 1

#Step 5 - Feature data development of the training set
----
The x and y are fit to a Logistic regression model from sklearn, using three fold cross validation

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegressionCV(cv=3)
logreg.fit(x_value, y)

#Step 6 - Feature data development of the testing set 
----
The network features are calculated for the edges in the test set, taken from "test-public.csv".

Th pandas dataframe is transformed to a numpy array, which can be used by the logistic regression model.

In [ ]:
test_file_original = pd.read_csv("test-public.csv")
test_file_original.set_index("Id")

sample_set = pd.DataFrame()

for i in test_file_original.index:
    source = test_file_original["Source"][i]
    sink = test_file_original["Sink"][i]
    #sample_set = sample_set.set_value(i,"Source",source)
    #sample_set = sample_set.set_value(i,"Sink",sink)
    array = [(source,sink)]
    preds = nx.resource_allocation_index(G,array)
    for u,v,p in preds:
        ra_score = p
    sample_set = sample_set.set_value(i,"ra_score",ra_score)
    preds = nx.jaccard_coefficient(G,array)
    for u,v,p in preds:
        jc_score = p
    sample_set = sample_set.set_value(i,"jc_score",jc_score)
    preds = nx.preferential_attachment(G,array)
    for u,v,p in preds:
        pa_score = p
    sample_set = sample_set.set_value(i,"pa_score",pa_score)
    cn_num = len(sorted(nx.common_neighbors(G, source, sink)))
    sample_set = sample_set.set_value(i,"cn_num",cn_num)  
    preds = nx.adamic_adar_index(G,array)
    for u,v,p in preds:
        pa_score = p
    sample_set = sample_set.set_value(count,"aa_score",aa_score)

sample_set = sample_set[sorted(sample_set.columns)]
sample_value = sample_set.values


#Step 7 - Prediction of test data 
----
The Logistic regression model is used to predict the probability of an edge (Class 1) on the test edges. The results are loaded into the test file for submission.

In [ ]:
test_file_original = pd.read_csv("test-public.csv")
test_file_original.set_index("Id")

for i in sample_set.index:
    try:
        alpha = 0
        array = sample_values[i]
        score_nnet = logreg.predict_proba(np.array(array,ndmin=2))
        score_nnet = score_nnet[0][1]
        print(str(i) + "LR Score is " + str(score_nnet))
        test_file_original.set_value(i,"Predicted",score_nnet)
    except Exception as e:
        print(str(e))
test_file_original = test_file_original[["Id","Predicted"]]
test_file_original.to_csv("test_file_results_LR.csv")